In [ ]:
# An example call of the tomo_workflow function
# TODO -- should the includes be inside the function?
# includes
include("../modules/generate_raw_data.jl")
include("../modules/process_raw_data.jl")
include("../modules/geometry.jl")
include("../modules/scene.jl")
include("../modules/range_spread_function.jl") # as RSF
include("../modules/orbits.jl")
include("../modules/sync.jl")
include("../modules/error_sources.jl")
include("../modules/performance_metrics.jl")
include("../modules/antenna.jl")
include("../modules/simsetup.jl")
include("../modules/user_parameters.jl")
include("../modules/plotting.jl")


In [ ]:
# Next is to call and validate the input parameters
include("../inputs/predefined-input-parameters.jl")
params = UserParameters.inputParameters()

# Check consistency of input parameters
paramsIsValid = UserParameters.validateInputParams(params)

In [ ]:
include("../modules/tomo_workflow_function.jl")
# Here is the function call itself. Only the params is needed -- TODO: is this true?
image_3D = TomoWorkflow.generate_tomo(params);

In [ ]:
# Based on input settings, calculate performance metrics
# Calculate point target performance metrics
resolutions, PSLRs, ISLRs, loc_errors, scene_axis11,scene_axis22,scene_axis33, PSF_metrics = Performance_Metrics.computePTPerformanceMetrics(image_3D, params)
println("Resolutions: ",round.(resolutions,digits=8)," in scene axes units")
println("Location Errors: ",round.(loc_errors,digits=8)," in scene axes units")
println("PSLRs: ",round.(PSLRs,digits=2)," dB")
println("ISLRs: ",round.(ISLRs,digits=2)," dB")
println("PSF Peak Amplitude: ",round(maximum(20*log10.(image_3D)),digits=2)," dB")

In [ ]:
# Plotting a tomogram and/or cuts

# Plots (1D PSF cuts are displayed by default in the performance.metrics module)
if params.display_geometry || params.display_RSF_rawdata || params.display_input_scene || params.display_tomograms != 0
    const display_geometry_coord_txt=Plotting.coordinates(params.display_geometry_coord)
    const ts_coord_txt=Plotting.coordinates(params.ts_coord_sys)
    if params.display_RSF_rawdata; Plotting.plot_RSF_rawdata(ft, t_rx, MF, Srx, Np, Nst, rawdata, params); end
    if params.display_geometry
        # convert platform and target locations to desired coordinate system
        const p_loc,t_loc,s_loc=Geometry.convert_platform_target_scene_coordinates(Np,Nst,Nt,p_xyz,t_xyz_3xN,targets_loc,s_xyz_3xN,s_loc_3xN,avg_peg, params)
        Plotting.plot_geometry(orbit_time,orbit_pos,p_loc,t_loc,s_loc,display_geometry_coord_txt)
    end
    if params.display_input_scene; Plotting.plot_input_scene(inputscene_3D, ts_coord_txt, params);end
    if params.display_tomograms != 0; Plotting.plot_tomogram(image_3D, ts_coord_txt, scene_axis11, scene_axis22, scene_axis33, PSF_metrics, params);end
    if params.display_input_scene; Plotting.plot_input_scene(diff_image3D, ts_coord_txt, params);end
end

In [ ]:
# Example loop over aperture start times and calc metrics over partial orbit
aperture_time_vec = 0 : 100 : 1000
resolution_results = Array{Float64}(undef,length(aperture_time_vec))

for i = 1 : length(aperture_time_vec)
    aperture_time = aperture_time_vec[i]
    params = UserParameters.inputParameters(SAR_start_time = aperture_time, user_defined_orbit=0)
    image_3D = TomoWorkflow.generate_tomo(params);
    
    #calc metrics
    resolutions, PSLRs, ISLRs, loc_errors, scene_axis11,scene_axis22,scene_axis33, PSF_metrics = Performance_Metrics.computePTPerformanceMetrics(image_3D, params)
    resolution_results[i] = resolutions
end#aperture_time

plot(aperture_time_vec, resolution_results)